In [1]:
from td_api import Account
from datetime import datetime, timedelta
import numpy as np
import pandas as pd

data = pd.read_csv("temp.csv")


In [4]:
columns = [
    "symbol",
    "bidPrice",
    "bidSize",
    "askPrice",
    "askSize",
    "lastPrice",
    "quoteTimeInLong",
    "strikePrice",
    "contractType",
    "underlying",
    "delta",
    "gamma",
    "theta",
    "vega",
    "rho",
    "underlyingPrice"
    "expiration"
]
data["expiration"] = pd.to_datetime(data[["expirationYear", "expirationMonth", "expirationDay"]])
data[columns]

Index(['symbol', 'assetType', 'assetMainType', 'cusip', 'description',
       'bidPrice', 'bidSize', 'askPrice', 'askSize', 'lastPrice', 'lastSize',
       'openPrice', 'highPrice', 'lowPrice', 'closePrice', 'netChange',
       'totalVolume', 'quoteTimeInLong', 'tradeTimeInLong', 'mark',
       'openInterest', 'volatility', 'moneyIntrinsicValue', 'multiplier',
       'digits', 'strikePrice', 'contractType', 'underlying', 'expirationDay',
       'expirationMonth', 'expirationYear', 'daysToExpiration', 'timeValue',
       'deliverables', 'delta', 'gamma', 'theta', 'vega', 'rho',
       'securityStatus', 'theoreticalOptionValue', 'underlyingPrice',
       'uvExpirationType', 'exchange', 'exchangeName', 'lastTradingDay',
       'settlementType', 'netPercentChangeInDouble', 'markChangeInDouble',
       'markPercentChangeInDouble', 'delayed'],
      dtype='object')


KeyError: ('delta', 'strikePrice', 'symbol')

In [6]:

df = {}

for i in range(3):
    sub_df = {}
    for j in range(10):
        sub_df["sub_%d" % j] = np.random.random(20)
    df["df_%d" % i] = sub_df

df = pd.DataFrame(df)
print(df)

Index(['sub_0', 'sub_1', 'sub_2', 'sub_3', 'sub_4', 'sub_5', 'sub_6', 'sub_7',
       'sub_8', 'sub_9'],
      dtype='object')
